In [1]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset_path = 'E:/Omdena/experiments'
final_df = pd.DataFrame([])

In [3]:
#experiments=os.listdir(dataset_path)

for exp_num in sorted(os.listdir(dataset_path), key=int):
    #dataframe for csv files
    file_path=os.path.join(dataset_path,exp_num,'training_data.csv')
    csv_df=pd.read_csv(file_path,parse_dates=['time'])
    csv_df.drop(['device-id'],axis=1,inplace=True)
    
    #dataframe for json metadata files
    json_path=os.path.join(dataset_path,exp_num,'metadata.json')
    json_df=pd.DataFrame([pd.read_json(json_path, typ='series')])
    #json_df.drop(['start_time'],axis=1,inplace=True)
    
    #replacing the '_' with '-' to maintain uniform naming convention
    new_cols=dict(zip(json_df.columns.to_list(),[i.replace('-','_') for i in json_df.columns]))
    json_df.rename(columns=new_cols, inplace=True)
    new_cols=dict(zip(csv_df.columns.to_list(),[i.replace('-','_') for i in csv_df.columns]))
    csv_df.rename(columns=new_cols, inplace=True)
    
    # merge two dataframes
    temp_key = 'temp' # create tempory column only for merge
    csv_df[temp_key]=1
    json_df[temp_key]=1
    merged_df = csv_df.merge(json_df, how='outer', on=temp_key).drop(temp_key, axis=1)

    # take important columns to left most side
    #first_cols = ['id', 'sensor_node_id', 'specimen_class', 'gas_scan_cnt']
    #rearrange_cols = first_cols + merged_df.columns.drop(first_cols).to_list()
    #merged_df = merged_df.loc[:, rearrange_cols]
    
    '''Encode target classes as clean_sir=0 and in_smoke=1'''
    
    merged_df["specimen_class"].replace({"clean_air": 0, "in_smoke": 1}, inplace=True)
  
    #rename columns
    merged_df = merged_df.rename(columns={'id': 'experiment', 'time': 'scan_time', 'specimen_class': 'encoded_specimen','burn_material_amount':'burn_material_amount(g)',
                                         'gas_scan_temp_0': 'temp_0','gas_scan_temp_1': 'temp_1','gas_scan_temp_2': 'temp_2','gas_scan_temp_3': 'temp_3','gas_scan_temp_4': 'temp_4',
                                         'gas_scan_temp_5': 'temp_5','gas_scan_temp_6': 'temp_6','gas_scan_temp_7': 'temp_7','gas_scan_temp_8': 'temp_8','gas_scan_temp_9': 'temp_9',
                                         'gas_scan_humid_0': 'humid_0','gas_scan_humid_1': 'humid_1','gas_scan_humid_2': 'humid_2','gas_scan_humid_3': 'humid_3','gas_scan_humid_4': 'humid_4','gas_scan_humid_5': 'humid_5',
                                         'gas_scan_humid_6': 'humid_6', 'gas_scan_humid_7': 'humid_7','gas_scan_humid_8': 'humid_8','gas_scan_humid_9': 'humid_9' })

    final_df = final_df.append(merged_df, ignore_index=True)
# convert datatype
final_df.gas_scan_cnt = final_df.gas_scan_cnt.astype('int')


In [4]:
final_df

,sensor_node_id,scan_time,temp_0,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,...,trigger,burn_material,burn_material_amount(g),end_time,hotplate_start,hotplate_temp,experiment,sensor_hotplate_distance,start_time,venue
0,146,2021-09-01 09:01:36.817528009+00:00,18.8,19.0,19.6,19.6,19.7,20.0,20.1,20.1,...,manual,SP2,600g,2021-09-01T18:50:46Z,2021-09-01T10:03:57Z,500,6,30m,2021-09-01T08:30:22Z,hall
1,146,2021-09-01 09:02:13.375891+00:00,19.2,19.3,19.8,19.8,19.9,20.2,20.2,20.2,...,manual,SP2,600g,2021-09-01T18:50:46Z,2021-09-01T10:03:57Z,500,6,30m,2021-09-01T08:30:22Z,hall
2,146,2021-09-01 09:02:50.057426+00:00,19.3,19.4,19.9,19.9,20.0,20.3,20.3,20.3,...,manual,SP2,600g,2021-09-01T18:50:46Z,2021-09-01T10:03:57Z,500,6,30m,2021-09-01T08:30:22Z,hall
3,146,2021-09-01 09:03:26.565743+00:00,19.3,19.5,20.0,20.0,20.1,20.3,20.4,20.3,...,manual,SP2,600g,2021-09-01T18:50:46Z,2021-09-01T10:03:57Z,500,6,30m,2021-09-01T08:30:22Z,hall
4,146,2021-09-01 09:04:03.635421991+00:00,19.4,19.5,20.0,20.0,20.1,20.4,20.4,20.4,...,manual,SP2,600g,2021-09-01T18:50:46Z,2021-09-01T10:03:57Z,500,6,30m,2021-09-01T08:30:22Z,hall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,130,2021-09-09 13:54:01.448236942+00:00,25.4,25.5,26.1,26.1,26.2,26.4,26.5,26.5,...,manual,BM2,0.2g,2021-09-09T14:40:40Z,2021-09-09T12:56:39Z,250,15,4m,2021-09-09T12:00:24Z,chamber
2494,130,2021-09-09 13:54:38.105902910+00:00,25.5,25.6,26.1,26.2,26.3,26.5,26.6,26.6,...,manual,BM2,0.2g,2021-09-09T14:40:40Z,2021-09-09T12:56:39Z,250,15,4m,2021-09-09T12:00:24Z,chamber
2495,130,2021-09-09 13:55:14.770633935+00:00,25.5,25.7,26.2,26.2,26.3,26.6,26.6,26.6,...,manual,BM2,0.2g,2021-09-09T14:40:40Z,2021-09-09T12:56:39Z,250,15,4m,2021-09-09T12:00:24Z,chamber
2496,130,2021-09-09 13:55:51.865952014+00:00,25.6,25.7,26.3,26.3,26.4,26.6,26.6,26.6,...,manual,BM2,0.2g,2021-09-09T14:40:40Z,2021-09-09T12:56:39Z,250,15,4m,2021-09-09T12:00:24Z,chamber


In [5]:
final_df['sensor_node_id'].value_counts()

133    309
130    295
134    287
136    260
137    238
139    231
132    227
143    221
146    219
140    211
Name: sensor_node_id, dtype: int64

In [6]:
date_cols=['scan_time','start_time','end_time','hotplate_start']

#converting the above mentioned columns datatypes to date_time
for cols in date_cols:
    final_df[cols]=pd.to_datetime(final_df[cols],utc=True)

In [7]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2498 entries, 0 to 2497
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   sensor_node_id            2498 non-null   int64              
 1   scan_time                 2498 non-null   datetime64[ns, UTC]
 2   temp_0                    2498 non-null   float64            
 3   temp_1                    2498 non-null   float64            
 4   temp_2                    2498 non-null   float64            
 5   temp_3                    2498 non-null   float64            
 6   temp_4                    2498 non-null   float64            
 7   temp_5                    2498 non-null   float64            
 8   temp_6                    2498 non-null   float64            
 9   temp_7                    2498 non-null   float64            
 10  temp_8                    2498 non-null   float64            
 11  temp_9           

In [8]:
final_df['hotplate_temp'].value_counts()

500          1099
250           627
open fire     428
320           344
Name: hotplate_temp, dtype: int64

Keeping hotplate_temp as string as it contains open fire values

In [9]:
final_df['burn_material_amount(g)'].value_counts()

600g      1363
n/a        428
0.1g       323
0.2g       208
0.3g       133
0.066g      43
Name: burn_material_amount(g), dtype: int64

burn_material_amount contains null values for the values where hotplate_temp is open fire. Checking class wise values for open fire.

In [10]:
is_na =  final_df['burn_material_amount(g)']=='n/a'
is_na.head()

0    False
1    False
2    False
3    False
4    False
Name: burn_material_amount(g), dtype: bool

In [11]:
burn_material_na = final_df[is_na]

In [12]:
burn_material_na['encoded_specimen'].value_counts()

1    331
0     97
Name: encoded_specimen, dtype: int64

In [13]:
#Replacing n/a values with NAN and converting the values to float
final_df["burn_material_amount(g)"].replace({"n/a": np.nan}, inplace=True)
final_df["burn_material_amount(g)"] = final_df["burn_material_amount(g)"].str.strip('g').astype(float)

In [14]:
final_df

,sensor_node_id,scan_time,temp_0,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,...,trigger,burn_material,burn_material_amount(g),end_time,hotplate_start,hotplate_temp,experiment,sensor_hotplate_distance,start_time,venue
0,146,2021-09-01 09:01:36.817528009+00:00,18.8,19.0,19.6,19.6,19.7,20.0,20.1,20.1,...,manual,SP2,600.0,2021-09-01 18:50:46+00:00,2021-09-01 10:03:57+00:00,500,6,30m,2021-09-01 08:30:22+00:00,hall
1,146,2021-09-01 09:02:13.375891+00:00,19.2,19.3,19.8,19.8,19.9,20.2,20.2,20.2,...,manual,SP2,600.0,2021-09-01 18:50:46+00:00,2021-09-01 10:03:57+00:00,500,6,30m,2021-09-01 08:30:22+00:00,hall
2,146,2021-09-01 09:02:50.057426+00:00,19.3,19.4,19.9,19.9,20.0,20.3,20.3,20.3,...,manual,SP2,600.0,2021-09-01 18:50:46+00:00,2021-09-01 10:03:57+00:00,500,6,30m,2021-09-01 08:30:22+00:00,hall
3,146,2021-09-01 09:03:26.565743+00:00,19.3,19.5,20.0,20.0,20.1,20.3,20.4,20.3,...,manual,SP2,600.0,2021-09-01 18:50:46+00:00,2021-09-01 10:03:57+00:00,500,6,30m,2021-09-01 08:30:22+00:00,hall
4,146,2021-09-01 09:04:03.635421991+00:00,19.4,19.5,20.0,20.0,20.1,20.4,20.4,20.4,...,manual,SP2,600.0,2021-09-01 18:50:46+00:00,2021-09-01 10:03:57+00:00,500,6,30m,2021-09-01 08:30:22+00:00,hall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,130,2021-09-09 13:54:01.448236942+00:00,25.4,25.5,26.1,26.1,26.2,26.4,26.5,26.5,...,manual,BM2,0.2,2021-09-09 14:40:40+00:00,2021-09-09 12:56:39+00:00,250,15,4m,2021-09-09 12:00:24+00:00,chamber
2494,130,2021-09-09 13:54:38.105902910+00:00,25.5,25.6,26.1,26.2,26.3,26.5,26.6,26.6,...,manual,BM2,0.2,2021-09-09 14:40:40+00:00,2021-09-09 12:56:39+00:00,250,15,4m,2021-09-09 12:00:24+00:00,chamber
2495,130,2021-09-09 13:55:14.770633935+00:00,25.5,25.7,26.2,26.2,26.3,26.6,26.6,26.6,...,manual,BM2,0.2,2021-09-09 14:40:40+00:00,2021-09-09 12:56:39+00:00,250,15,4m,2021-09-09 12:00:24+00:00,chamber
2496,130,2021-09-09 13:55:51.865952014+00:00,25.6,25.7,26.3,26.3,26.4,26.6,26.6,26.6,...,manual,BM2,0.2,2021-09-09 14:40:40+00:00,2021-09-09 12:56:39+00:00,250,15,4m,2021-09-09 12:00:24+00:00,chamber


Now splitting the data on the basis of sensors.

In [15]:
final_df.sensor_node_id.value_counts()

133    309
130    295
134    287
136    260
137    238
139    231
132    227
143    221
146    219
140    211
Name: sensor_node_id, dtype: int64

In [17]:
sensor_ids=list(final_df.sensor_node_id.value_counts().index)
sensor_ids

[133, 130, 134, 136, 137, 139, 132, 143, 146, 140]

In [18]:
train_ids=[133,130,134,136,137]
valid_ids=[139,132]
test_ids=[143,146,140]

In [19]:
train_set = final_df[final_df['sensor_node_id'].isin(train_ids)]
train_set.shape

(1389, 44)

In [20]:
valid_set = final_df[final_df['sensor_node_id'].isin(valid_ids)]
valid_set.shape

(458, 44)

In [21]:
test_set = final_df[final_df['sensor_node_id'].isin(test_ids)]
test_set.shape

(651, 44)

In [22]:
total_size = len(train_set) + len(valid_set) + len(test_set)
total_size, len(final_df)

(2498, 2498)

In [23]:
#Exporting the sets to csv files

train_set.to_csv('train_set_v2.csv')                
valid_set.to_csv('valid_set_v2.csv')
test_set.to_csv('test_set_v2.csv')